In [ ]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as  pd
import sklearn
import os
import time
from tensorflow import keras
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

In [ ]:
#导入数据
(x_train_all,y_train_all),(x_test,y_test) = keras.datasets.fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]
print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

In [ ]:
#图片归一化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaler = scaler.fit_transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaler = scaler.fit_transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [ ]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

In [ ]:
#tf.keras.Sequential()
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = [28,28]))
for _ in range(20):
    #selu激活函数，自带归一化
    model.add(keras.layers.Dense(100,
                                 activation="selu"))
#dropout  
# model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.AlphaDropout(rate=0.5)) # 优点：1.均值和方差不变，2.归一化的性质不变
model.add(keras.layers.Dense(10,
                            activation="softmax"))
#relu : y  = max(0,x)
#softmax: 将向量变成概率分布 x = [x1,x2,x3] y = [e^x1/sum,e^x2/sum,e^x3/sum] sum = e^x1+e^x2+e^x

#loss 如果lable 是一个one-hot 向量 ：categorical_croossentropy，不是向量：sparse_categorical_crossentropy
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = 'sgd',
             metrics = ["accuracy"])

In [ ]:
model.summary()

In [ ]:
logdir = r'C:\Users\Xiaoi\Desktop\tensorflow2.0\dnn_selu_dropout_callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "fashion_mnist_model.h5")

_callbacks = [
    keras.callbacks.TensorBoard(logdir),#tensotBoard 路径
    keras.callbacks.ModelCheckpoint(output_model_file,
                                   save_best_only=True), #保存最好的模型
    keras.callbacks.EarlyStopping(patience=5,
                                 min_delta=1e-3) # 5轮之内，最低为1e-3 就停止
]
history = model.fit(x_train_scaled,
                    y_train,
                   epochs=10,
                   validation_data=(x_valid_scaler,y_valid),
                   callbacks=_callbacks)

In [ ]:
def plot_learning_curver(history):
    pd.DataFrame(history.history).plot(figsize = (8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
plot_learning_curver(history)
"""
1.参数众多，训练不充分
2.梯度消失-》链式法则-》复合函数f(g(x))
批归一化 缓解梯度消失
"""
